In [1]:
from databroker.v2 import temp
from bluesky import RunEngine
from bluesky.plans import count
from bluesky.plan_stubs import mv
from secop_ophyd.secop_ophyd.SECoPDevices import SECoP_Node_Device, SECoPReadableDevice, SECoPMoveableDevice, SECoP_CMD_Device
import asyncio
import random



# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({})
db = temp()
RE.subscribe(db.v1.insert)

from bluesky.plans import scan
from ophyd.sim import det, motor

#Example of adding metadata to RE environment

investigation_id = "kmnk2n34"

RE.md["investigation_id"] = investigation_id

robo_dev = await SECoP_Node_Device.create('localhost','10770',RE.loop)

# now do something
RE(scan([det],motor,1,2,10),reason="for a demo")




ModuleNotFoundError: No module named 'secop_ophyd.secop_ophyd'

In [ ]:
await robo_dev.read_configuration()

storage:SECoPReadableDevice= robo_dev.storage

reading = await storage.value.read()

loadshort:SECoP_CMD_Device = storage.load_short_dev


reading = await storage.value.read()
print(reading)


loadshort:SECoP_CMD_Device = storage.load_short_dev

for samplepos in range(1,13):
    await loadshort.samplepos_arg.set(samplepos)
    await loadshort.substance_arg.set(random.randint(0,6))

    await loadshort.load_short_x.execute()

    await storage.wait_for_IDLE()
    print(samplepos)
        
        

In [ ]:
await robo_dev.storage.read()
#await robo_dev.storage.read_configuration()

RE(count([storage],delay=2,num=5))


In [ ]:
sample:SECoPMoveableDevice = robo_dev.sample


for i in range(1,13):
    #grab sample i and hold in Measurement Pos
    RE(mv(sample,i))
    
    # Start measuring program
    await sample.measure_dev.meaxure_x.execute()
    
    #subscribe to read values of simulated EPICs Device
    
    await sample.wait_for_IDLE()
    
    # put sample back into storage
    RE(mv(sample,0))